In [ ]:
import xarray as xr
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import panel as pn
import ipywidgets as ipw

import matplotlib
matplotlib.use('agg')

An interactive REPL or notebook interfaces are incredibly powerful tool for quickly doing exploratory analysis, however they generally still require manually changing arguments to method and function calls to see their effect. To further ease exploratory workflows hvPlot ships with a so called `interactive` API, which mirrors the regular API of your favorite data analysis libraries like Pandas, Dask, and xarray but makes it possible to replace constant arguments with widgets that dynamically update the output of the method calls and which tranparently chain, behaving just like the object that is being wrapped.

In this user guide we will explore how to use the interactive API on xarray and pandas objects:

In [ ]:
ds = xr.tutorial.load_dataset('air_temperature')
ds

## Interactive widgets

We can supply both regular values, widgets and parameters as arguments to methods on the `interactive` accessor. The repr of the resulting object will contain a layout of the widget and a view of the resulting output:

In [ ]:
slider = pn.widgets.IntSlider(name='time', start=0, end=10)

ds.air.interactive(width=800).isel(time=slider)

ipywidgets are also supported as dynamic arguments:

In [ ]:
slider = ipw.IntSlider(description='time', min=0, max=10)

ds.air.interactive(width=800).isel(time=slider)

We can also let interactive automatically populate a widget, which is particularly useful when working with `DiscreteSlider` widgets:

In [ ]:
ds.air.interactive(width=800).sel(time=pn.widgets.DiscreteSlider)

## Docstrings

When accessing a method on the `interactive` accessor it will transparently mirror the docstring of the equivalent method in the underlying library being wrapped:

In [ ]:
print(ds.air.interactive.isel.__doc__)

## Plotting

One of the most useful aspects of the interactive API is to feed the output of chained method calls into a plot.

### Matplotlib

The output can be almost anything, the HTML repr or a matplotlib plot:

In [ ]:
ds.air.interactive.sel(time=pn.widgets.DiscreteSlider).plot()

We can animate the output with a `Player` widget, and customize the location of the widget using the `loc` keyword argument to the `interactive` accessor:

In [ ]:
time = pn.widgets.Player(name='time', start=0, end=10, loop_policy='loop', interval=100)

ds.air.interactive(loc='bottom').isel(time=time).plot()

### hvPlot

We can also make use of the `.hvplot` method to get interactive plots:

In [ ]:
slider = pn.widgets.FloatSlider(name='quantile', start=0, end=1)

ds.air.interactive.quantile(slider, dim='time').hvplot(data_aspect=1)

You can chain any number of methods, with as many widgets controlling steps in this pipeline as you wish:

In [ ]:
q = pn.widgets.FloatSlider(name='quantile', start=0, end=1)

(ds
 .air
 .interactive(loc='left')
 .sel(time=pn.widgets.DiscreteSlider)
 .quantile(q=q, dim='lon')
 .hvplot(aspect=1)
)

We can also use a `RangeSlider` to select a slice and compute the mean:

In [ ]:
range_slider = pn.widgets.RangeSlider(start=0, end=len(ds.time), step=1)

(ds
 .air
 .interactive
 .isel(time=range_slider)
 .mean('time')
 .hvplot()
)

Note that `.interactive` can be chained arbitrarily, e.g. we can even convert to a dataframe using `to_dataframe` and then call pandas methods:

In [ ]:
lat = pn.widgets.DiscreteSlider(name='Latitude', options=sorted(ds.lat.values))

ds.air.interactive.sel(lat=lat).to_dataframe().groupby('time').mean().hvplot('time', 'air')

## Operators

You can apply math operators on the interactive object:

In [ ]:
slider = pn.widgets.IntSlider(name='time', start=0, end=10)

ds.air.interactive(width=800).isel(time=slider).mean().item() + 10

You can even do math with a widget:

In [ ]:
slider = pn.widgets.IntSlider(name='time', start=0, end=10)

offset = pn.widgets.IntSlider(name='Offset', start=0, end=10)

ds.air.interactive.isel(time=slider).mean().item() + offset

In [ ]:
((ds.air.interactive.sel(time=pn.widgets.DiscreteSlider) - ds.air.mean('time'))
  .hvplot(cmap='RdBu_r', clim=(-20, 20))
)

In [ ]:
kind = pn.widgets.Select(options=['contour', 'image', 'contourf'])

pn.panel((ds.air.interactive.sel(time=pn.widgets.DiscreteSlider) - ds.air.mean('time'))
 .hvplot(cmap='RdBu_r', clim=(-20, 20), kind=kind).layout
)